In [12]:
import agate

In [35]:
tester = agate.TypeTester(limit=100,force={
    'NRC_DESC': agate.Text(),
    'NRC_CODE': agate.Number(),
        'BEDSCODE': agate.Text(),
        'L1_COUNT': agate.Text(),
        'L1_PCT': agate.Text(),
        'L2_COUNT': agate.Text(),
        'L2_PCT': agate.Text(),
        'L3_COUNT': agate.Text(),
        'L3_PCT': agate.Text(),
        'L4_COUNT': agate.Text(),
        'L4_PCT': agate.Text(),
        'L2-L4_PCT': agate.Text(),
        'L3-L4_PCT': agate.Text(),
        'MEAN_SCALE_SCORE': agate.Text(),
        'COUNTY_DESC': agate.Text(),
        'TOTAL_TESTED': agate.Text(),
        'COUNTY_CODE': agate.Text()
})

In [36]:
school_16 = agate.Table.from_csv('3-8_ELA_MATH_2016.csv', column_types=tester)

/Users/lheinle/anaconda/envs/py35/lib/python3.5/site-packages/agate/table/__init__.py:104: DuplicateColumnWarning: Column name "" already exists in Table. Column will be renamed to "_2".
  warn_duplicate_column(new_column_name, final_column_name)


In [38]:
print(school_16)

|--------------------+------------|
|  column            | data_type  |
|--------------------+------------|
|  SY_END_DATE       | Date       |
|  NRC_CODE          | Number     |
|  NRC_DESC          | Text       |
|  COUNTY_CODE       | Text       |
|  COUNTY_DESC       | Text       |
|  BEDSCODE          | Text       |
|  NAME              | Text       |
|  ITEM_SUBJECT_AREA | Text       |
|  ITEM_DESC         | Text       |
|  dian              | Number     |
|  SUBGROUP_NAME     | Text       |
|  TOTAL_TESTED      | Text       |
|  L1_COUNT          | Text       |
|  L1_PCT            | Text       |
|  L2_COUNT          | Text       |
|  L2_PCT            | Text       |
|  L3_COUNT          | Text       |
|  L3_PCT            | Text       |
|  L4_COUNT          | Text       |
|  L4_PCT            | Text       |
|  ...               | ...        |
|--------------------+------------|



In [68]:
erie_niagara = school_16.where(lambda row: row['COUNTY_DESC'] in ['ERIE', 'NIAGARA'])
schools_total = erie_niagara.where(lambda row: row['SUBGROUP_NAME'] in ['All Students'])

In [89]:
print(len(erie_niagara.rows))
print(len(schools_total.rows))
for row in schools_total.rows[:10]:
    print(row['NAME'])
    print(row['ITEM_DESC'])
    print(row['L3_COUNT'])

25544
1861
ERIE COUNTY
Grade 3 ELA
2315
ERIE COUNTY
Grade 3 Math
1479
ERIE COUNTY
Grade 4 ELA
1441
ERIE COUNTY
Grade 4 Math
1487
ERIE COUNTY
Grade 5 ELA
1276
ERIE COUNTY
Grade 5 Math
1415
ERIE COUNTY
Grade 6 ELA
1189
ERIE COUNTY
Grade 6 Math
1115
ERIE COUNTY
Grade 7 ELA
1168
ERIE COUNTY
Grade 7 Math
1081


In [96]:
def calculate_counts(row):
    try:
        l3_counts = int(row['L3_COUNT'])
    except ValueError:
        l3_counts = 0
    try:
        l4_counts = int(row['L4_COUNT'])
    except ValueError:
        l4_counts = 0
    profficent = l3_counts + l4_counts
    return profficent

In [97]:
erie_calcs = schools_total.compute([
        ('pass_count', agate.Formula(agate.Number(), lambda row: calculate_counts(row)))
    ])

In [106]:
percent_calculates = {}
for row in erie_calcs.rows:
    percent_calculates[row['NAME']] = {'math': {'totals': [], 'profficent': [], 'classes': []}, 'ela': {'totals': [], 'profficent': [], 'classes': []}}
print(percent_calculates['WILLIAM STREET SCHOOL'])

{'ela': {'totals': [], 'classes': [], 'profficent': []}, 'math': {'totals': [], 'classes': [], 'profficent': []}}


In [107]:
for row in erie_calcs.rows:
    if row['ITEM_SUBJECT_AREA'] == 'ELA':
        data_dict = percent_calculates[row['NAME']]['ela']
    else:
        data_dict = percent_calculates[row['NAME']]['math']
    try:
        data_dict['totals'].append(int(row['TOTAL_TESTED']))
    except ValueError:
        data_dict['totals'].append(0)
    try:
        data_dict['classes'].append(row['ITEM_DESC'])
    except ValueError:
        data_dict['classes'].append(0)
    try:
        data_dict['profficent'].append(int(row['pass_count']))
    except ValueError:
        data_dict['profficent'].append(0)
print(percent_calculates['WILLIAM STREET SCHOOL'])     

{'ela': {'totals': [189, 201, 197], 'classes': ['Grade 4 ELA', 'Grade 5 ELA', 'Grade 6 ELA'], 'profficent': [94, 78, 89]}, 'math': {'totals': [185, 196, 191], 'classes': ['Grade 4 Math', 'Grade 5 Math', 'Grade 6 Math'], 'profficent': [118, 127, 107]}}


In [121]:
for school, values in percent_calculates.items():
    print(school)
    print(values)
    total_sum = sum(values['ela']['totals'])
    print('total sum = {0}'.format(sum(values['ela']['totals'])))
    print(values['ela']['profficent'])
    profficient_sum = sum(values['ela']['profficent'])
    print('profficient sum = {0}'.format(sum(values['ela']['profficent'])))
    try:
        correct_value = ((profficient_sum / total_sum) * 100)
        percent_calculates[school]['ela']['total_percent'] = float("{0:.1f}".format(correct_value))
    except ZeroDivisionError:
        percent_calculates[school]['ela']['total_percent'] = '-'
    total_math_sum = sum(values['math']['totals'])
    print('total sum = {0}'.format(sum(values['math']['totals'])))
    print(values['math']['profficent'])
    profficient_math_sum = sum(values['math']['profficent'])
    print('profficient sum = {0}'.format(sum(values['math']['profficent'])))
    try:
        correct_math_value = ((profficient_math_sum / total_math_sum) * 100)
    except ZeroDivisionError:
        
    percent_calculates[school]['math']['total_percent'] = float("{0:.1f}".format(correct_math_value))

WILLIAM STREET SCHOOL
{'ela': {'totals': [189, 201, 197], 'total_percent': 44.46, 'classes': ['Grade 4 ELA', 'Grade 5 ELA', 'Grade 6 ELA'], 'profficent': [94, 78, 89]}, 'math': {'totals': [185, 196, 191], 'total_percent': 61.54, 'classes': ['Grade 4 Math', 'Grade 5 Math', 'Grade 6 Math'], 'profficent': [118, 127, 107]}}
total sum = 587
[94, 78, 89]
profficient sum = 261
total sum = 572
[118, 127, 107]
profficient sum = 352
NORTH PARK JUNIOR HIGH SCHOOL
{'ela': {'totals': [223, 165], 'total_percent': 27.58, 'classes': ['Grade 7 ELA', 'Grade 8 ELA'], 'profficent': [47, 60]}, 'math': {'totals': [196, 124], 'total_percent': 15.0, 'classes': ['Grade 7 Math', 'Grade 8 Math'], 'profficent': [38, 10]}}
total sum = 388
[47, 60]
profficient sum = 107
total sum = 320
[38, 10]
profficient sum = 48
NIAGARA FALLS CITY SCHOOL DISTRICT
{'ela': {'totals': [490, 459, 454, 457, 487, 453], 'total_percent': 22.86, 'classes': ['Grade 3 ELA', 'Grade 4 ELA', 'Grade 5 ELA', 'Grade 6 ELA', 'Grade 7 ELA', 'Grade

ZeroDivisionError: division by zero

In [122]:
print(percent_calculates['WILLIAM STREET SCHOOL'])  

{'ela': {'totals': [189, 201, 197], 'total_percent': 44.5, 'classes': ['Grade 4 ELA', 'Grade 5 ELA', 'Grade 6 ELA'], 'profficent': [94, 78, 89]}, 'math': {'totals': [185, 196, 191], 'total_percent': 61.5, 'classes': ['Grade 4 Math', 'Grade 5 Math', 'Grade 6 Math'], 'profficent': [118, 127, 107]}}
